 # <center>PSK Transceiver Demo</center>

This notebook simulates the PSK transceiver.

In [2]:
%matplotlib inline

import sys

import ipywidgets as widgets
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

import sksdr
import sksdr.psk_trans
import utils

trans = None
tx_frame_idx = 0
rx_frame_idx = 0
#fig_stats = []
#gs_stats = []

def execute(b):
    global trans, tx_frame_idx, rx_frame_idx #, fig_stats, gs_stats
    args = dict()
    for idx, (key, widget) in enumerate(param_widgets_dict.items()):
        args[key] = widget.value
    args['scrambler_poly'] = eval(args['scrambler_poly'])
    args['scrambler_init_state'] = eval(args['scrambler_init_state'])
    if args['modulation'] == sksdr.BPSK:
        args['mod_symbols'] = [0, 1]
    else: # QPSK
        args['mod_symbols'] = [0, 1, 3, 2]
    
    if trans is None:
        trans = sksdr.psk_trans.PSKTrans(**args)
    
    fs = args['sample_rate']
    fs_rx = fs / args['downsampling']
    ts = 1 / fs
    ts_rx = 1 / fs_rx
    msg = msg_widget.value
    num_frames = num_frames_widget.value
    
    for i in range(num_frames):
        tx_msg = msg + ' ' + '{:03d}'.format(tx_frame_idx)
        rx_msg = msg + ' ' + '{:03d}'.format(rx_frame_idx)
        tx_ret = trans.transmit(tx_msg)
        chan_frame = trans.channel(tx_ret['frame'], tx_frame_idx)
        rx_ret = trans.receive(chan_frame, rx_msg)
        
        # Tx plots
        with tx_accord_widget.children[0]:
            tx_accord_widget.set_title(0, 'Transmitter Signals: Frame ' + str(tx_frame_idx))
            tx_accord_widget.children[0].clear_output(wait=True)
            fig_tx = plt.figure(figsize=(15,10))
            gs_tx = gridspec.GridSpec(2, 2, figure=fig_tx)
            sksdr.time_plot([tx_ret['frame'].real, tx_ret['frame'].imag], ['Re', 'Im'], [ts, ts], 'Transmit Signal', fig=fig_tx, gs=gs_tx[0, :])
            sksdr.psd_plot(tx_ret['frame'], fs, 'Transmit Signal PSD', fig=fig_tx, gs=gs_tx[1, 0])
            sksdr.scatter_plot(tx_ret['symbols'], 'Transmit Signal Constellation', fig=fig_tx, gs=gs_tx[1, 1])
            fig_tx.tight_layout()
            plt.show(fig_tx)
    
        # Rx plots
        with rx_accord_widget.children[0]:
            rx_accord_widget.set_title(0, 'Receiver Signals: Frame ' + str(tx_frame_idx))
            rx_accord_widget.children[0].clear_output(wait=True)
            fig_rx = plt.figure(figsize=(15,30))  
            gs_rx = gridspec.GridSpec(6, 2, figure=fig_rx)
            sksdr.time_plot([chan_frame.real, chan_frame.imag], ['Re', 'Im'], [ts, ts], 'Receive Signal', fig=fig_rx, gs=gs_rx[0, :])
            sksdr.time_plot([chan_frame.real, rx_ret['agc_frame'].real], ['Before Re', 'After Re'], [ts, ts], 'AGC Signal', fig=fig_rx, gs=gs_rx[1, :])
            sksdr.psd_plot(rx_ret['rx_filter_down_frame'], fs/2, 'Before CFC PSD', fig=fig_rx, gs=gs_rx[2, 0])
            sksdr.psd_plot(rx_ret['cfc_frame'], fs_rx, 'After CFC PSD', fig=fig_rx, gs=gs_rx[2, 1])
            sksdr.psd_plot(rx_ret['cfc_frame'], fs_rx, 'Before Fsync PSD', fig=fig_rx, gs=gs_rx[3, 0])
            sksdr.psd_plot(rx_ret['fsync_frame'], fs_rx, 'After Fsync PSD', fig=fig_rx, gs=gs_rx[3, 1])
            sksdr.time_plot([rx_ret['fsync_frame'].real, rx_ret['ssync_frame'].real], ['Before Re', 'After Re'], [ts_rx, ts_rx * 2], 'Symbol sync', fig=fig_rx, gs=gs_rx[4, :])
            if rx_ret['valid']:
                sksdr.time_plot([rx_ret['phase_amb_frame'].real, rx_ret['phase_amb_frame'].imag], ['Re', 'Im'], [ts_rx * 2, ts_rx * 2], 'Received Frame', fig=fig_rx, gs=gs_rx[5, :])
            fig_rx.tight_layout()
            plt.show(fig_rx)

        # Stats/output  
        with stats_accord_widget.children[0]:
            out = stats_accord_widget.children[0]
            out.append_stdout('\nSending frame: ' + str(tx_frame_idx) + '\n')
            out.append_stdout('\tmsg: ' + str(tx_msg) + '\n')
            out.append_stdout('\nReceiving frame: ' + str(rx_frame_idx) + '\n')
            if rx_ret['valid']:
                rx_ber = rx_ret['BER']
                out.append_stdout('\tmsg: ' + str(rx_ret['rx_msg_ascii']) + '\n')
                out.append_stdout('\tBER: ' + '{}/{}'.format(str(rx_ber[0]),str(rx_ber[1])) + '\n')
                #fig_stats.append(plt.figure(figsize=(15,5)))
                #fig_stats = plt.figure(figsize=(15,5))
                #gs_stats.append(gridspec.GridSpec(1, 2, figure=fig_stats[-1]))
                #gs_stats = gridspec.GridSpec(1, 2, figure=fig_stats)
                #time_plot([rx_ret['agc_error']], ['Error'], [ts], 'AGC Error', fig=fig_stats, gs=gs_stats[0, :])
                #plt.show(fig_stats)
                out.append_stdout('\tCFC offset: ' + str(rx_ret['cfc_offset']) + '\n')
                out.append_stdout('\tPreamble indexes (last sample): ' + str(rx_ret['prb_end_idxs']) + '\n')
                rx_frame_idx += 1
            else:
                out.append_stdout('\tNo valid preamble detected or not enough bits in frame\n')
        tx_frame_idx += 1

def reset(b=None):    
    global trans, tx_frame_idx, rx_frame_idx
    tx_accord_widget.set_title(0, 'Transmitter Signals')
    tx_accord_widget.children[0].clear_output()
    rx_accord_widget.set_title(0, 'Receiver Signals')
    rx_accord_widget.children[0].clear_output()
    stats_accord_widget.children[0].clear_output()
    trans = None
    tx_frame_idx = 0
    rx_frame_idx = 0
    # fig_stats.clear()
    # gs_stats.clear()


# Widgets common style settings
style = dict(utils.description_width_style)

# Settings grids and widgets
param_widgets_dict = dict()
tx_grid = widgets.GridspecLayout(4, 3)
tx_grid[0, 0] = param_widgets_dict['frame_size'] = widgets.IntText(value=100, continuous_update=False, description='Frame size (Symbols):', style=style)
tx_grid[0, 1] = param_widgets_dict['sample_rate'] = widgets.IntText(value=200e3, continuous_update=False, description='Sample rate (Hz):', style=style)
tx_grid[0, 2] = param_widgets_dict['modulation'] = widgets.Dropdown(options=[('BPSK', sksdr.BPSK), ('QPSK', sksdr.QPSK)], value=sksdr.QPSK, continuous_update=False, description='Modulation:', style=style)
tx_grid[1, 0] = param_widgets_dict['upsampling'] = widgets.IntText(value=4, continuous_update=False, description='Upsampling:', style=style)
tx_grid[1, 1] = param_widgets_dict['scrambler_poly'] = widgets.Text(value='[1, 1, 1, 0, 1]', continuous_update=False, description='Scrambler polynomial:', style=style)
tx_grid[1, 2] = param_widgets_dict['scrambler_init_state'] = widgets.Text(value='[0, 1, 1, 0]', continuous_update=False, description='Scrambler initial state:', style=style)
tx_grid[2, 0] = param_widgets_dict['rrc_rolloff'] = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.5, continuous_update=False, description='RRC rolloff:', style=style)
tx_grid[2, 1] = param_widgets_dict['rrc_span'] = widgets.IntSlider(min=1, max=120, step=1, value=10, continuous_update=False, description='RRC span (symbols):', style=style)
tx_grid[3, :] = msg_widget = widgets.Textarea(value='Hello world!', continuous_update=False, description='Message:', style=style, layout=widgets.Layout(height='auto', width='auto'))

chan_grid = widgets.GridspecLayout(2, 3)
chan_grid[0, 0] = param_widgets_dict['chan_snr'] = widgets.BoundedFloatText(value=15, min=-1000, max=1000, continuous_update=False, description='SNR (dB):', style=style)
chan_grid[0, 1] = param_widgets_dict['chan_delay_type'] = widgets.Dropdown(options=[('Triangle', 'triangle')], continuous_update=False, description='Delay type:', style=style)
chan_grid[0, 2] = param_widgets_dict['chan_delay_step'] = widgets.BoundedFloatText(value=0.05, min=0, max=np.finfo(float).max, continuous_update=False, description='Delay step (samples):', style=style)
chan_grid[1, 0] = param_widgets_dict['chan_max_delay'] = widgets.BoundedFloatText(value=8, min=0, max=np.finfo(float).max, continuous_update=False, description='Max delay (samples):', style=style)
chan_grid[1, 1] = param_widgets_dict['chan_freq_offset'] = widgets.BoundedFloatText(value=5e3, min=0, max=np.finfo(float).max, continuous_update=False, description='Frequency offset (Hz):', style=style)
chan_grid[1, 2] = param_widgets_dict['chan_phase_offset'] = widgets.BoundedFloatText(value=47, min=0, max=np.finfo(float).max, continuous_update=False, description='Phase offset (degrees):', style=style)

rx_grid = widgets.GridspecLayout(5, 3)
rx_grid[0, 0] = param_widgets_dict['downsampling'] = widgets.IntText(value=2, continuous_update=False, description='Downsampling:', style=style)
rx_grid[0, 1] = param_widgets_dict['agc_ref_power'] = widgets.FloatSlider(value=1/param_widgets_dict['upsampling'].value, min=0.0, max=10.0, step=1, continuous_update=False, description='AGC ref power:', style=style)
rx_grid[0, 2] = param_widgets_dict['agc_max_gain'] = widgets.FloatSlider(value=60.0, min=0.0, max=80.0, step=5.0, continuous_update=False, description='AGC max gain (dB):', style=style)
rx_grid[1, 0] = param_widgets_dict['agc_det_gain'] = widgets.FloatSlider(value=0.01, min=0.01, max=0.05, step=0.01, continuous_update=False, description='AGC detector gain:', style=style)
rx_grid[1, 1] = param_widgets_dict['agc_avg_len'] = widgets.IntText(value=100, min=10, max=150, step=1, continuous_update=False, description='AGC averaging (samples):', style=style)
rx_grid[1, 2] = param_widgets_dict['coarse_freq_comp_res'] = widgets.FloatText(value=25, min=1, max=100, step=1, continuous_update=False, description='Coarse freq comp resolution (Hz):', style=style)
rx_grid[2, 0] = param_widgets_dict['fsync_damp_factor'] = widgets.FloatSlider(value=1, min=0.01, max=1.5, step=0.01, continuous_update=False, description='Frequency sync damp factor:', style=style)
rx_grid[2, 1] = param_widgets_dict['fsync_norm_loop_bw'] = widgets.FloatSlider(value=0.01, min=0.01, max=0.2, step=0.01, continuous_update=False, description='Frequency sync norm bw:', style=style)
rx_grid[2, 2] = param_widgets_dict['ssync_K'] = widgets.FloatSlider(value=1, min=0.01, max=1.5, step=0.01, continuous_update=False, description='Symbol sync K:', style=style)
rx_grid[3, 0] = param_widgets_dict['ssync_A'] = widgets.FloatSlider(value=1 / np.sqrt(2), min=0.01, max=1.5, step=0.01, continuous_update=False, description='Symbol sync A:', style=style)
rx_grid[3, 1] = param_widgets_dict['ssync_damp_factor'] = widgets.FloatSlider(value=1, min=0.01, max=1.5, step=0.01, continuous_update=False, description='Symbol sync damp factor:', style=style)
rx_grid[3, 2] = param_widgets_dict['ssync_norm_loop_bw'] = widgets.FloatSlider(value=0.01, min=0.01, max=0.2, step=0.01, continuous_update=False, description='Symbol sync norm bw:', style=style)
rx_grid[4, 0] = param_widgets_dict['prb_det_thr'] = widgets.FloatSlider(value=8, min=1, max=16, step=0.01, continuous_update=False, description='Preamble detector threshold:',style=style)

# Settings tab widget
settings_tab_widget = widgets.Tab(children=[tx_grid, chan_grid, rx_grid])
for i, v in enumerate(['Tx Settings', 'Channel Settings', 'Rx Settings']):
    settings_tab_widget.set_title(i, v)

# Buttons widgets
reset_button_widget = widgets.Button(description='Reset', tooltip='Reset states')
reset_button_widget.on_click(reset)
execute_button_widget = widgets.Button(description='Execute', tooltip='Execute simulation')
execute_button_widget.on_click(execute)
num_frames_widget = widgets.BoundedIntText(description='Number of frames:', value=2, min=1, max=100, continuous_update=False, style=style)

tx_accord_widget = widgets.Accordion(children=[widgets.Output()])
rx_accord_widget = widgets.Accordion(children=[widgets.Output()])
stats_accord_widget = widgets.Accordion(children=[widgets.Output()])
stats_accord_widget.set_title(0, 'Stats/Output')

# Main layout
ui = widgets.VBox([
        # Settings
        settings_tab_widget,
        # Buttons
        widgets.HBox([execute_button_widget, reset_button_widget, num_frames_widget]),
        # Plots and output
        tx_accord_widget,
        rx_accord_widget,
        stats_accord_widget
])
reset()
display(ui)
utils.display_hacks()

HTML(value='\n        <style>\n        .jupyter-widgets-output-area .output_scroll {\n                height: …

<IPython.core.display.Javascript object>

DEBUG:sksdr.freq_sync:FSYNC init: phase_recovery_loop_bw=0.020000, phase_recovery_gain=2.000000, theta=0.008000, d=1.016064, p_gain=0.007874, i_gain=0.000031
DEBUG:sksdr.symbol_sync:SSYNC init: theta=0.004000, d=-5.443286, p_gain=-0.002939, i_gain=-0.000012
DEBUG:sksdr.psk_trans:[ 0.02001273+0.02285199j  0.017583  +0.03502993j -0.02883508-0.02860071j
  0.0459965 +0.04613029j -0.151652  -0.07915925j -0.01608327+0.00419194j
  0.10350937-0.16181578j -0.11164268+0.01051985j  0.00786191+0.21723945j
  0.19248753+0.03780473j]
DEBUG:sksdr.psk_trans:[-6.43482568e-06-7.34775063e-06j  1.11162271e-04+1.69626142e-04j
 -2.24606446e-05-9.24733973e-05j -3.28519138e-04-2.16904507e-05j
  6.56733037e-04+2.98092778e-05j -5.17136056e-04+8.52340462e-04j
  4.25491122e-04-1.11581099e-03j -2.00059059e-04-2.95339864e-04j
 -3.98469082e-03-1.07752937e-03j  1.23656621e-02+4.67309829e-03j]
DEBUG:sksdr.psk_trans:[-6.43482568e-06-7.34775063e-06j  1.58056150e-04+1.27076087e-04j
 -7.24451478e-05-6.17050267e-05j -2.1114